In [ ]:
from math import log
from random import random


class Base(dict):
    """Container for Stochastic Simulation Algorithms (SSA)"""

    def __init__(
        self,
        initial_conditions,
        propensities,
        stoichiomemtry
    ):
        """Initialize SSA"""
        self._propen = sorted(propensities.items(), reverse=True)
        self._stoich = sorted(stoichiometry.items(), reverse=True)
        self._exclud = dict(propen=list(), stoich=list())
        super().__init__(**initial_conditions)
        
    def direct(self):
        """Indefinite generator of direct-method trajectories"""
        while True:
            while not self.exit():
                
                # init step: evaluate propensities and partition
                weights = list((k, v(self)) for k,v in self.propen)
                partition = sum(tup[1] for tup in weights)
                
                # monte carlo step 1: next reaction time
                sojourn = log(1.0 / random()) / partition
                self["time"].append(
                    self["time"][-1] + sojourn
                )
                
                # monte carlo step 2: next reaction
                partition = partition * random()
                j = len(weights) - 1
                while partition >= 0.0:
                    partition -= weights.pop()[1]
                    j -= 1
                    
                # final step: update reaction species
                for species, delta in self.stoich[j][1].items():
                    self[species].append(
                        self[species][-1] + delta
                    )
                    
            yield self
            for key in self:
                del self[key][1:]
        
    def first_reaction(self):
        """Indefinite generator of 1st-reaction trajectories"""
        while True:
            while not self.exit():

                # monte carlo step: generate reaction times
                times = list(
                    (k,  log(1.0 / random()) / v(model))
                    for k,v in self.propen
                ).sort(key=lambda t: t[1])

                # update next reaction time
                model["time"].append(times[0][1])

                # update reaction species
                reaction_stoich = self.stoich[times[0][0]]
                for species, delta in reaction_stoich:
                    self[species] += delta
                    
            yield self
            for key in self:
                del self[key][1:]
            
    def exit(self, *args, **kwargs):
        """Break out of trajectory on conditions"""
        raise NotImplementedError
        
    @property
    def propen(self):
        """Return valid propensities"""
        for k in range(len(self._propen)):
            rxn, pro = self._propen[k]
            if pro(self) <= 10**(-8):
                self._exclud["propen"].append(
                    self._propen.pop(k)
                )
        for k in range(len(self._exclud["propen"])):
            rxn, pro = self._exclud["propen"][k]
            if pro(self) > 10**(-8):
                self._propen.append(
                    self._exclud["propen"].pop(k)
                )
        return self._propen.sort(reverse=True)
        
    @property
    def stoich(self):
        """Return valid stoichiometries"""
#         for key in self:
#             for k in range(len(self._stoich)):
#                 rxn, sto = self._stoich[k]
#                 if self[key] + sto.get(key, 0) < 0:
#                     self._exclud["stoich"].append(
#                         self._stoich.pop(k)
#                     )
#             for k in range(len(self._exclud["stoich"])):
#                 rxn, sto = self._exclud["stoich"][k]
#                 if self[key] > 10**(-8):
#                     self._propen.append(
#                         self._exclud["stoich"].pop(k)
#                     )
        return self._stoich.sort(reverse=True)
        

In [ ]:
class Epidemic(Base):
    """Epidemic without vital dynamics"""
    
    def exit(self):
        if self["i"] == 0:
            if self["r"][-1] == 300:
                return True
            elif self["s"][-1] > 0:
                self.reset()
                return False
        return False

In [ ]:
# initial species counts and sojourn times
initital_conditions = {
    "s": [290],
    "i": [10],
    "r": [0],
    "time": [0.0],
}


# propensity functions
propensities = {
    0: lambda d: 3.0 * d["s"][-1] * d["i"][-1] / 300,
    1: lambda d: 5.0 * d["i"][-1],
}


# change in species for each propensity
stoichiometry = {
    0: {"s": -1, "i": 1, "r": 0},
    1: {"s": 0, "i": -1, "r": 1},
}


In [ ]:
from matplotlib import pyplot, rcParams


# make figure 10" x 3", 200 dots per inch
# rcParams["figure.figsize"] = 10, 3
# rcParams["figure.dpi"] = 200


# # instantiate figure and axes
# figure, axes = pyplot.subplots(1, 3)


# append trajectories to plot
epidemic = Epidemic(
    initital_conditions,
    propensities,
    stoichiometry
)
trajectories = 0
for trajectory in epidemic.direct():
    pyplot.plot(trajectory["time"], trajectory["s"])
    pyplot.plot(trajectory["time"], trajectory["i"])
    pyplot.plot(trajectory["time"], trajectory["r"])
    trajectories += 1
    if trajectories == 1:
        break
pyplot.show()

In [ ]:
# initial species counts and sojourn times
state = {
    "s": [290],
    "i": [10],
    "r": [0],
    "time": [0.0],
}


# propensity functions
propensities = {
    0: lambda d: 5.0 * d["s"][-1] * d["i"][-1] / 300,
    1: lambda d: 3.0 * d["i"][-1],
}


# change in species for each propensity
stoichiometry = {
    0: {"s": -1, "i": 1, "r": 0},
    1: {"s": 0, "i": -1, "r": 1},
}




while True:

    # init step: evaluate propensities and partition
    weights = list(
        (k, v(state)) for k,v in propensities.items()
    )
    partition = sum(weight[1] for weight in weights)

    # monte carlo step 1: next reaction time
    sojourn = log(1.0 / random()) / partition
    state["time"].append(
        state["time"][-1] + sojourn
    )

    # monte carlo step 2: next reaction
    partition = partition * random()
    while partition >= 0.0:
        j, weight = weights.pop(0)
        partition -= weight
        
    # final step: update reaction species
    for species, delta in stoichiometry[j].items():
        state[species].append(
            state[species][-1] + delta
        )